# Cryptography with RSA Protocol

Alice has a problem. She would like Bob to send her an encrypted  message using the techniques from Lecture 3 of Week 10. However her computer only generates primes $p$ of at most $16$ binary bits (i.e. $p < 2^{16}$). She still wants a public key $(N,e)$ where ($e = 65537$ and) $N$ is a composite integer of at least $1024$ bits... as she thinks that this will be secure. So she decides to compute several (for example 15-30) primes of at most $16$ bits, and to randomly use one or more of each of these primes to make up the prime decomposition of an integer of bit length at least $1024$. She will then, as in the RSA protocol,  publish her public key $(N,e)$. 

**Reminder.** In the RSA protocol Alice would have had private key $(p,q)$ where $p$ and $q$ are primes such that $N = p \cdot q$ (i.e. $p^1 \cdot q^1$ is the prime decompositon of $N$). Here Alice will have as private key the dictionary that encodes the prime decomposition of $N$. So her key will be of the form 
$\{p_1\text{:}\;e_1, p_2\text{:}\; e_2, \dots ,p_m\text{:}\; e_m\}$ where $\prod^m_{i=1} p_i^{e_i}$ is the prime decomposition of $N$.  

What Alice does not realise is that her modification to the RSA protocol makes it highly insecure. Indeed Eve, who has been eavesdropping (i.e. listening in) on the line of transmission between Bob and Alice, thinks she can now hack the ciphertext integer $c$ that Bob transmits to Alice so as to extract Bob's original message. 

**Note 1.** Bob proceeds precisely: given a message in the form of a string he uses the function `convert_to_integer` to convert this string to an integer $m$. He then computes the ciphertext   

$$
c \,=\, m^e \;\left( \mathrm{mod}\; N  \right)\, 
$$

and transmits $c$ to Alice (via the line on which Eve is eavesdropping). 

Eve needs a function that, on input `N`, outputs the prime decomposition of `N`. She decides to develop her own algorithm instead of using the function `decompose`. Her algorithm does the following. 

1. It starts by setting variables `current_N` = `N` and `current_decomp` = `{}` (the empty  dictionary).   
2. It iterates through stages `p` = $2,3,4,5,\dots$ as follows. 
3. At stage `p` if it is not the case that `current_N` > 1 then the algorithm returns `current_decomp`. 
4. Otherwise at stage `p`, if `p` is prime, and `current_N` is divisible by `p` then, during stage `p` - supposing that `e` is the greatest integer such that `current_N` is divisible by `p**e` - the value of `current_N` is divided by `p**e` and the key, value pair `p: e` is added to `current_decomp`. The algorithm then proceeds to stage `p + 1`. 
5. If at stage `p` neither the conditions in 3 hold nor those in 4 hold, then nothing is done and the algorithm proceeds directly to stage `p + 1`. 

We will implement Eve's algorithm as the function `eves_decompose`. Our function will take as input a positive integer `N` and output the dictionary representing the prime decomposition of `N`. 
Our function should use the primality test `is_prime`.



In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def is_prime(n,verbose=False): 
    '''
    Checks whether the argument n is a prime number using a brute force 
    search for factors between 1 and n. We made it verbose here for 
    illustration. (I.e. it prints out its results.)
    '''
    # First, 1 is not prime.
    if n == 1:
        return False
    # If n is even then it is only prime if it is 2
    if n % 2 == 0: 
        if n == 2: 
            return True
        else:
            if verbose:
                print("{} is not prime: {} is a factor. ".format(n,2))
            return False
    # So now we can consider odd numbers only. 
    j = 3
    rootN = n**0.5
    # Now check all numbers 3,5,... up to sqrt(n)
    while j <= rootN: 
        if n % j == 0:
            if verbose:
                print("{} is not prime: {} is a factor.".format(n,j))
            return False
        j = j + 2
    if verbose:
        print("{} is prime.".format(n))
    return True 



In [29]:
# You will need the following three imported functions in this homework
# from cryptography_functions import is_prime, modular_inverse, convert_to_text

# The following two lines of code give you an overview of the content of the 
# module cryptography_functions as a printout 
# import cryptography_functions 
# help(cryptography_functions)

# You can also simply inspect the file cryptography_functions.py in jupyter

In [30]:
def eves_decompose(N):
# first set variables as required in part 1
    current_N = N
    current_decomp = {}
# iterate through p = 2,3,4,5.. etc
    for p in range(2,N):
# at stage p, if it is not the case that current_N > 1, return current_decomp
        if current_N <= 1:
            return current_decomp
# Otherwise at stage p, if p is prime, and current_N is divisible by p then, 
# during stage p - supposing that e is the greatest integer such that current_N 
# is divisible by p**e - the value of current_N is divided by p**e and the key,
# value pair p: e is added to current_decomp. The algorithm then proceeds to stage p + 1.
        if is_prime(p) == True:
            if pow(current_N,1,p) == 0:
                power = False
                e = 2
                while not power:
                    if pow(current_N,1,pow(p,e)) == 0:
                        e += 1   
                    else:  
                        e -= 1
                        power = True
                current_N = current_N // p**e
                current_decomp[p] = e  
    return current_decomp

In [31]:
# This cell is for your  own tests and rough work
# Here's an easy example. 
K = 3**4 * 17**2 * 23*5 * 37*1
eves_decompose(K)

{3: 4, 5: 1, 17: 2, 23: 1, 37: 1}

In [32]:
# Testing area, using a 1030 bit number N as input  
N = int('''
83706564861599899653596475840424730458044600907771647007334050372336
09892280960094075716733849713226033323457029628024552980324599333370
00350684058261595028326433198358461905686764348015800175804034215489
64422648032179381441885225061637776302818897970437444034711022068655
35792022518811024903672288999022155493'''.replace('\n',''))
N_dict = {8273: 6, 9173: 4, 10847: 4, 11311: 4, 11621: 4, 11953: 1, 
          12479: 2, 12923: 3, 13331: 4, 17209: 6, 17827: 1, 22643: 3, 
          25693: 2, 28279: 5, 30059: 3, 40763: 1, 41149: 4, 42589: 6, 
          55529: 1, 56299: 3, 56659: 5}
# Your function should pass this test (and a hidden test)
assert eves_decompose(N) == N_dict
# There is one hidden test using a similarly large number as input

In [33]:
def gcd_ext(a,b):
    """Outputs (gcd,x,y) such that gcd=ax+by."""
    if not(a%1 ==0 and b%1==0):                         #Reject if trying to use for non-integers
        print( "Need to use integers for gcd.")
        return None
    if a == 0:                                          #Base case is when a=0.
        return (abs(b), 0, abs(b)//b)                   #Then gcd =|b| and is 0*a+1*b or 0*a-1*b. Use abs(b)//b
    else:
        quot=b//a                                       #The rule is that g=gcd(a,b)=gcd(b%a,a).
                                                        #Let b=qa+r where r=b%a
        g, x, y = gcd_ext(b%a, a)                       #And if  g=x1*r + y1*a then since r=b-qa
        return (g, y - quot * x, x)                     #We get g = a*(y1-q*x1)+x1*b.

In [34]:
def modular_inverse(a,b): 
    '''
    Given input (a,b) with a and b integers returns 
    the multiplicative inverse of a modulo b provided 
    gcd(a,b) = 1. Otherwise returns an error message.
    '''
    ic_message = 'The numbers are not comprime'
    (g,x,y) = gcd_ext(a,b)
    if not g == 1: 
        print(ic_message)
        return None
    x = x % b 
    return x 

In [35]:
# Function from Lecture 3 of Week 10 for your information. 
def rsa_decrypt(c,p,q,N,e): 
    '''
    Given input (c,p,q,N,e) returns the RSA decryption of ciphertext
    c using private key (p,q) and public key (N,e). (We input N as a 
    parameter to avoid having to recompute N = p*q.)
    '''
    totient = N - (p + q) + 1       # This is (p-1)*(q-1)
    f = modular_inverse(e,totient)  # Note: f * e = 1 (mod totient)
    return pow(c,f,N)               # This is m = c**f (mod N) where c = m**e

If Bob transmits $c \,=\, m^e \;\left( \mathrm{mod}\; N  \right)\,$ and $N = p \cdot q$ where $p$ and $q$ are prime, then `rsa_decrypt(c,p,q,N,e)` computes the integer $m$. Eve's idea is to modify this function so that it works for any composite positive number $N$ by replacing the two input parameters $p$, $q$ by a dictionary representing the prime decomposition of $N$. 

We will implement Eve's idea as the function `eves_decrypt` which takes as input (`c`, `N_decomp`, `N`, `e`) where `c` is the transmitted ciphertext, (`N`,`e`) is Alice's public key and `N_decomp` is the dictionary representing the prime decomposition of `N`. Under the assumption that the first input has value $c \,=\, m^e \;\left( \mathrm{mod}\; N  \right)\,$ our function will return $m$.

**Note 1.** We assume that the input is as described in the preceding paragraph. I.e$.$ our function does not need to handle incorrect input. 

In [36]:
def eves_decrypt(c,N_decomp,N,e):
    t = 1
# now calculate totient
    for i in N_decomp.keys():
        t = t * i**(N_decomp[i]-1) * (i-1)
# now calculate modular inverse
    f = modular_inverse(e,t)
    return pow(c,f,N)

In [37]:
# This cell is for your  own tests and rough work
# An easy example its easy to come up with others... 
N = 3**4 * 13**6 * 19**2 * 53**8 
N_decomp = eves_decompose(N)
e = 65537 
m = 53458936789543654   
c = pow(m,e,N)
m_new = eves_decrypt(c,N_decomp,N,e)
m_new # This should be the same as m (test this using an assert statement)

53458936789543654

In [38]:
# Testing area using large integers. 
# The ciphertext transmitted by Bob is c = m**e (mod N1) for some m. 
c = int('''
44040993505419961079485909296703212068236418038487210630216568513457921
38336402714823913994482821220533646724844746539049498179119183080513149
31408838044424151820320215981324447571417572264574845615546729136893812
49131244618204208416492118915226940262437591117873293250397449419056765
862894569167485210222637280212955530851785230'''.replace('\n',''))
# Alice's public key is (N1,e). 
e = 65537
N1 = int('''
10230611115167627701967833246600529852209634787210723189242636200865741
18444371403714267935391853062750090843150978074427150943772945195279171
00280972327473328434991888940897882646502852180501590304145861856226481
21545694830420218027450318764061548271599311224935464351312228384098752
0276059163143000753961230876956414639120841839'''.replace('\n',''))
# The prime decomposition of N1 is N1_decomp. 
N1_decomp = {8237: 1, 8377: 6, 10343: 1, 12941: 4, 14563: 3, 14939: 1, 
            15991: 7, 16691: 5, 20011: 1, 20627: 1, 22039: 5, 22111: 2,
            25633: 7, 26189: 5, 32603: 6, 50159: 7, 52259: 3, 54721: 3, 
            62383: 7}
m_decrypt = 30786253411008024821942804353038798977485124256255665198
# This is true - i.e. c = m_decrypt**e (mod N1) - since m_decrypt == m
assert c == pow(m_decrypt,e,N1)
# This is the test of your function 
assert eves_decrypt(c,N1_decomp,N1,e) == m_decrypt

In [39]:
def convert_to_text(number): 
    """ 
    Returns a string that is the decoding of the input integer number.
    This is done by converting number to a binary string, removing the 
    leading character '1', slicing out each 8 bit substring consecutively,
    converting each such string to the character it encodes and concatenating
    these characters to obtain the decoded string.    
    """
    # Remove '0b1' from the string
    bin_string = bin(number)[3:]    
    text = ''                           
    length = len(bin_string)
    for i in range(0,length,8):  
        # Pick out binary strings, 8 bits at a time
        byte_string = bin_string[i:i+8]   
        # Convert byte_string to a character before 
        # appending it to text 
        text = text + chr(int(byte_string,2))  
    return text

Eve has now got the necessary tools to try to hack Bob's ciphertext $c$, given that the key $(N,e)$ is public. Her idea is to develop an algorithm that uses the functions `eves_decompose` and `eves_decrypt` to compute $m$ and then converts $m$ back in to the original message using `convert_to_text`. 

We will implement this as the function `eves_hack` which takes as inputs either  (`c`, `N`, `e`, `verbose`) or simply (`c`, `N`, `e`) where `c` is the ciphertext,  (`N`,`e`) is Alice's public key and `verbose` is a boolean variable used as a switch.  Our function will return the original message sent by Bob as a string. 

When the optional input parameter `verbose` is set to `True` the function will produce a nicely formatted print out showing the various elements used and processed during the computation.

In [40]:
# calculate prime decomposition
def eves_hack(c,N,e,verbose=False):
    N_decomp = eves_decompose(N)
# find m
    m = eves_decrypt(c,N_decomp,N,e)
# find message
    message = convert_to_text(m)
    if verbose:
        print('Firstly:\nc = {},\nN = {},\ne = {}'.format(c,N,e))
        print('\nWe then decompsed N to a dictionary of its primes, {}'.format(N_decomp))
        print('\nNext we decrypted c by calculating its totient, thus finding m={}'.format(m))
        print('\nUsing the convert_to_text algorithm, we transfered m into string, giving the message:')
    return message

In [41]:
# The output of this cell - i.e. the printout generated by your function 
# (since verbose=True below) will be hand graded.

# The ciphertext transmitted by Bob is c2. 
c2 = int('''
13765857727387973232479930349817413548909598342616602193701631284325656267
80701134620002785255145626349800021032002509266271558856266934655194975687
59307106770371716547021307786879941777280361623176540814670229006368058329
84911920670968761587938063429816954180041385102041952823773836621293529317
2747559925006181429813854'''.replace('\n',''))

# Alice's public key is (N2,e). 
e = 65537
N2 = int('''
45826872776221747553965304038569114157321898032045936109324798759092548505
31613377090447584756132759059985898756678241036420212547809537827143541861
23768838634031792685862961283953201133968017357971449925795092740936013316
34869263468578803287892075018530064166921208113922227263664667767248577655
3724508783300940735066371
'''.replace('\n',''))

# Eve now hacks ciphertext c2
eves_hack(c2,N2,e,True)

Firstly:
c = 137658577273879732324799303498174135489095983426166021937016312843256562678070113462000278525514562634980002103200250926627155885626693465519497568759307106770371716547021307786879941777280361623176540814670229006368058329849119206709687615879380634298169541800413851020419528237738366212935293172747559925006181429813854,
N = 458268727762217475539653040385691141573218980320459361093247987590925485053161337709044758475613275905998589875667824103642021254780953782714354186123768838634031792685862961283953201133968017357971449925795092740936013316348692634685788032878920750185300641669212081139222272636646677672485776553724508783300940735066371,
e = 65537

We then decompsed N to a dictionary of its primes, {8573: 3, 10993: 1, 11699: 4, 12227: 3, 12487: 2, 12503: 3, 12697: 1, 15271: 4, 16993: 3, 19507: 3, 23371: 1, 23473: 3, 23599: 3, 25639: 1, 25841: 4, 27043: 4, 29723: 2, 32353: 4, 32503: 5, 33923: 1, 38237: 3, 38273: 1, 42899: 1, 49603: 3, 60527: 5, 62549: 5}

Next we decryp

"In 1931 Gödel had proved that arithmetic was incomplete, and this result would be the topic of Newman's final lectures."